In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch
import pandas as pd
from functools import partial
from src.models.autoencoder import Autoencoder
from src.data.utils import collate_batch
from src.data.load import load_data
from src.models.utils import *
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

### 1. Data

In [ ]:
# Hyperparameters
embed_dim = 256
batch_size = 64
EPOCHS = 10

In [7]:
train_dataset, val_dataset, test_dataset = load_data("../data/processed/")
tokenizer = get_tokenizer("spacy", language="en_core_web_sm")
vocab = torch.load("../data/vocab.pt")

train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    collate_fn=partial(collate_batch, vocab=vocab, tokenizer=tokenizer),
    sampler=train_dataset.get_sampler(),
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    collate_fn=partial(collate_batch, vocab=vocab, tokenizer=tokenizer),
    sampler=val_dataset.get_sampler(),
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=partial(collate_batch, vocab=vocab, tokenizer=tokenizer),
)

### 2. Training

In [9]:
model = Autoencoder(len(vocab), embed_dim, epochs=EPOCHS)

train_autoencoder(model, train_dataloader, val_dataloader)

| epoch   1 |  1000/ 2795 batches | loss    0.000
| epoch   1 |  2000/ 2795 batches | loss    0.000
-----------------------------------------------------------
| end of epoch   1 | time:  7.57s | valid accuracy    0.000 
-----------------------------------------------------------
| epoch   2 |  1000/ 2795 batches | loss    0.000
| epoch   2 |  2000/ 2795 batches | loss    0.000
-----------------------------------------------------------
| end of epoch   2 | time:  7.77s | valid accuracy    0.000 
-----------------------------------------------------------
| epoch   3 |  1000/ 2795 batches | loss    0.000
| epoch   3 |  2000/ 2795 batches | loss    0.000
-----------------------------------------------------------
| end of epoch   3 | time:  7.53s | valid accuracy    0.000 
-----------------------------------------------------------
| epoch   4 |  1000/ 2795 batches | loss    0.000
| epoch   4 |  2000/ 2795 batches | loss    0.000
---------------------------------------------------------

### 3. Evaluation

In [5]:
print('Checking the results of test dataset.')
accu_test = evaluate_autoencoder(model, test_dataloader, CrossEntropyLoss())
print('test accuracy {:8.8f}'.format(accu_test))

Checking the results of test dataset.
test accuracy 0.07424341


### 4. Export

In [6]:
#torch.save(model.state_dict(), "./autoencoder.pt")